In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("GSAF5.xlsx")

In [3]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [4]:
# Goal: Commit the perfect shark murder
# Problem Statement: Identify the correct location, activity,  time of the day and what time of the year

# Hypothesis 1: The perfect location is Australia (can be narrowed down on state level if fitting to the country)
# Hypothesis 2: The perfect time of the year is early in the year on a morning
# Hypothesis 3: The deadliest sharks will be white, tiger and bull

# Columns we need: "Fatal Y/N", "Country", "Date", "Activity",  "Time"
# -> GFM: Fatal & Country
# -> LB: Date
# -> MB: Activty & Time


# further stuf:
# - age, gender of the person we try to kill
# species for "e.g should be killed by white shark"





In [4]:
pd.set_option('display.max_columns', None)

df

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-02-22 00:00:00,2025.0,Unprovoked,Australia,Western Australia,Blasck Wall Reach Swan River Perth,Kayaking,Georgina Smith,F,35,None kayak badly damaged by shark,N,0815hrs,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-02-22 00:00:00,2025.0,Questionable,Australia,NSW,Off Port Stephens near Newcastle,Fishing was pulled of boat by shark,Paul Barning,M,58,Body not recovered,Y,1300hrs,Mako shark,Simon De Marchi: Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-22 00:00:00,2025.0,Unprovoked,Australia,Queensland,The Wrecks walking track Moreton Island,Snorkeling,Unknown Male,M,29,Abdominal and leg injuries,N,1503hrs,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-10 00:00:00,2025.0,Unprovoked,Bahamas,NaN,Bimini Bay,Swimming,Summer Layman,F,24,Left foot injured,N,1830hrs,bull shark,Dail Mail Australia Todd Smith: Kevin McMurray...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-02-10 00:00:00,2025.0,Unprovoked,Bahamas,NaN,Bimini Bay,Swimming,Rileigh Decker,F,20,Severed archilles tendon artery severed nerves...,N,1830hrs,bull shark,Dail Mail Australia Todd Smith: Kevin McMurray...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
7000,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
7001,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
7002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


In [6]:
def clean_date_column(df, column_name, date_format='%Y-%m-%d', fill_na=None):
    """
    Cleans a date column in a DataFrame and returns both cleaned and invalid rows.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the date column.
    - column_name (str): The name of the date column to clean.
    - date_format (str, optional): The format to standardize the date to (default: '%Y-%m-%d').
    - fill_na (str or pd.Timestamp, optional): A default date to fill missing/invalid values. If None, does not fill.

    Returns:
    - pd.DataFrame: The DataFrame with the cleaned date column.
    - pd.DataFrame: A DataFrame containing rows that had invalid dates.
    - dict: A summary of how many rows were affected.
    """
    df = df.copy()  # Avoid modifying the original DataFrame

    # Convert column to datetime, coercing errors to NaT
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')

    # Separate valid and invalid date rows
    valid_df = df[df[column_name].notna()].copy()
    invalid_df = df[df[column_name].isna()].copy()

    # Count invalid dates
    invalid_count = len(invalid_df)

    # Handle missing values in valid_df
    if fill_na is not None:
        valid_df[column_name] = valid_df[column_name].fillna(pd.Timestamp(fill_na))

    # Standardize date format
    valid_df[column_name] = valid_df[column_name].dt.strftime(date_format)

    # Summary of changes
    summary = {
        "total_rows": len(df),
        "valid_rows": len(valid_df),
        "invalid_rows": invalid_count,
        "filled_rows": invalid_count if fill_na else 0
    }

    print("Date Cleaning Summary:", summary)
    
    return valid_df, invalid_df, summary

cleaned_df, invalid_df, summary = clean_date_column(df, 'Date')

print("\nCleaned DataFrame:")
display(cleaned_df)

print("\nInvalid DataFrame:")
display(invalid_df)


Date Cleaning Summary: {'total_rows': 7004, 'valid_rows': 6129, 'invalid_rows': 875, 'filled_rows': 0}

Cleaned DataFrame:


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-02-22,2025.0,Unprovoked,Australia,Western Australia,Blasck Wall Reach Swan River Perth,Kayaking,Georgina Smith,F,35,None kayak badly damaged by shark,N,0815hrs,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-02-22,2025.0,Questionable,Australia,NSW,Off Port Stephens near Newcastle,Fishing was pulled of boat by shark,Paul Barning,M,58,Body not recovered,Y,1300hrs,Mako shark,Simon De Marchi: Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-22,2025.0,Unprovoked,Australia,Queensland,The Wrecks walking track Moreton Island,Snorkeling,Unknown Male,M,29,Abdominal and leg injuries,N,1503hrs,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-10,2025.0,Unprovoked,Bahamas,NaN,Bimini Bay,Swimming,Summer Layman,F,24,Left foot injured,N,1830hrs,bull shark,Dail Mail Australia Todd Smith: Kevin McMurray...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-02-10,2025.0,Unprovoked,Bahamas,NaN,Bimini Bay,Swimming,Rileigh Decker,F,20,Severed archilles tendon artery severed nerves...,N,1830hrs,bull shark,Dail Mail Australia Todd Smith: Kevin McMurray...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6853,1721-06-01,1721.0,Unprovoked,ITALY,Sardinia,"Ponte della Maddelena,",Swimming,male,M,NaN,"FATAL, partial remains recovered from shark’s...",Y,NaN,"White shark, 1600-lb female",F. Ricciardi; A. De Maddalena.,1721.06.00-Maddalena.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1721.06.00,1721.06.00,150.0,NaN,NaN
6854,1703-03-26,1703.0,Unprovoked,BARBADOS,Southwest coast,Carlisle Bay,Swimming,"Samuel Jennings, a deserter from the British f...",M,19,"Hand and foot severely bitten, surgically ampu...",N,Night,NaN,"W.R.Cutter, Vol.1, p.252",1703.03.26-Jennings.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1703.03.26,1703.03.26,149.0,NaN,NaN
6859,1691-01-01,1691.0,Unprovoked,CANADA,Quebec,Gaspé,NaN,Sailor,M,NaN,FATAL,Y,NaN,NaN,Canadian Shark Attack Registry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6865,1970-01-01,1595.0,Unprovoked,INDIA,Kerala,River Cochin,Ship lay at anchor & man was working on its ru...,male,M,NaN,"Leg severed mid-thigh, hand severed, arm above...",UNKNOWN,NaN,NaN,The Voyage of John Huyghen van Linschoten,1595.00.00-Cochin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1595.00.00,1595.00.00,140.0,NaN,NaN



Invalid DataFrame:


,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
83,NaT,2023.0,Questionable,EGYPT,Southern Red Sea,Fury Shoal,Scuba diving,Gary Virgo,M,66,No injury. No attack; the shark simply head-b...,N,NaN,Oceanic whitetip shark,"Daily Mail, 11/2/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaT,2023.0,Unprovoked,PHILIPPINES,Bolinao,"Pangasinan, Ilocos Norte, Luzon Island",Fishing,male,M,NaN,Laceration to left thigh,M,NaN,NaN,"Manila Bulletin, 9 /10/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaT,2023.0,Watercraft,Coral Sea,NaN,NaN,On a round-the-world expedition,9 m inflatable catamaran,M,NaN,Catamaran severely damaged by sharks. No injur...,N,01h00,Cookiecutter sharks,"B. Myatt, GSAF",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaT,2023.0,Watercraft,SOUTH AFRICA,Eastern Cape Province,Chinta East,Watching the sardine run,crew,NaN,NaN,"No injury to occupants, shark defated boat",N,NaN,NaN,"Ladbible, 7/14/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,NaT,2023.0,Unprovoked,USA,Hawaii,Laysan,Filming,Netflix crew,M,NaN,Sharks bit their inflatable boats,N,NaN,Tiger sharks,"BBC, 6/14/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
7000,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
7001,NaT,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
7002,NaT,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


-----
# Guillermo Working on Cleaning  Fatal Y/N


In [10]:
# Code to clean the 'Fatal Y/N' column in the DataFrame

# Define a dictionary to map current values to 'Yes' or 'No'
'''replacements = {
    'Y': 'Yes',
    'N': 'No',
    'UNKNOWN': 'No',  # Example of treating unknown as 'No', adjust as needed
    'nan': 'No',      # Handle missing values, adjust if they should be 'Yes' 
    # Add more mappings if necessary
}
'''
# Sample data as a list
values = ['N', 'Y', 'F', 'M', float('nan'), 'n', 'Nq', 'UNKNOWN', 2017, 'Y x 2', ' N', 'N ', 'y']

# Define a dictionary for the mappings
mapping = {
    'N': 'N',
    'Y': 'Y',
    'F': 'U',           # Assuming 'F' and 'M' are unusual entries we'll consider unknown ('U')
    'M': 'U',
    'n': 'N',
    'Nq': 'N',
    'UNKNOWN': 'U',
    2017: 'U',         # With the numerical could Keep numerical years unchanged or convert to 'U'
    'Y x 2': 'Y',       # Considering 'Y x 2' as a form of 'Yes'
    ' N': 'N',
    'N ': 'N',
    'y': 'Y'
}

# Apply the mapping to the column
df['Fatal Y/N'] = df['Fatal Y/N'].replace(mapping).fillna('U')

# Print the cleaned DataFrame column
# print(df['Fatal Y/N'].value_counts())


''' Sample data as a list
values = ['N', 'Y', 'F', 'M', float('nan'), 'n', 'Nq', 'UNKNOWN', 2017, 'Y x 2', ' N', 'N ', 'y']

different values in column fatal y/n:
 Fatal Y/N
N          4877
Y          1474
UNKNOWN      71
 N            7
F             5
M             3
n             1
Nq            1
2017          1
Y x 2         1
N             1
y             1
Name: count, dtype: int64
['N' 'Y' 'F' 'M' nan 'n' 'Nq' 'UNKNOWN' 2017 'Y x 2' ' N' 'N ' 'y']
Number of NaN values in 'Fatal Y/N' column: 561

The result is:

different values in column fatal y/n:
 Fatal Y/N
N    4887
Y    1476
U     641
Name: count, dtype: int64
['N' 'Y' 'U']
Number of NaN values in 'Fatal Y/N' column: 0'''




" Sample data as a list\nvalues = ['N', 'Y', 'F', 'M', float('nan'), 'n', 'Nq', 'UNKNOWN', 2017, 'Y x 2', ' N', 'N ', 'y']\n\ndifferent values in column fatal y/n:\n Fatal Y/N\nN          4877\nY          1474\nUNKNOWN      71\n N            7\nF             5\nM             3\nn             1\nNq            1\n2017          1\nY x 2         1\nN             1\ny             1\nName: count, dtype: int64\n['N' 'Y' 'F' 'M' nan 'n' 'Nq' 'UNKNOWN' 2017 'Y x 2' ' N' 'N ' 'y']\nNumber of NaN values in 'Fatal Y/N' column: 561\n\nThe result is:\n\ndifferent values in column fatal y/n:\n Fatal Y/N\nN    4887\nY    1476\nU     641\nName: count, dtype: int64\n['N' 'Y' 'U']\nNumber of NaN values in 'Fatal Y/N' column: 0"

----
# Cleaning Country:

In [11]:
# Strip leading/trailing whitespaces and convert to lowercase
df['Country'] = df['Country'].str.lower().str.strip()


In [12]:
# Replace NaN in the 'Country' column with 'unknown'
df['Country'] = df['Country'].fillna('unknown')


In [15]:
# 1st step: Clean the 'Country' column

# Create the 'Region' column because too many entries with regions instead of countries in the 'Country' column. 
# that way we can still use the data even if it more generalized. Also there are a lot of small islands that have to be research to know where are they located.

regions = [
    'atlantic ocean', 'pacific ocean', 'indian ocean', 'red sea', 'mediterranean sea',
    'caribbean sea', 'bay of bengal', 'gulf of aden', 'persian gulf', 'tasman sea',
    'south china sea', 'north atlantic ocean', 'south atlantic ocean', 'north pacific ocean',
    'south pacific ocean', 'southwest pacific ocean', 'central pacific', 'coral sea',
    'mid atlantic ocean', 'mid-pacifc ocean', 'between portugal & india', 'coast of africa',
    'the balkans', 'british isles', 'west indies', 'antarctica', 'ocean', 'africa',
    'europe', 'north america', 'south america', 'oceania', 'middle east', 'asia'
]
# Clean 'Country' by removing '?' and whitespace
df['Country_clean'] = df['Country'].str.replace('?', '', regex=False).str.strip()

# Initialize 'Region' as 'unknown'
df['Region'] = 'unknown'

# Identify rows where cleaned 'Country' is a region
is_region = df['Country_clean'].isin(regions)

# Update 'Region' and 'Country' for these rows
df.loc[is_region, 'Region'] = df.loc[is_region, 'Country_clean']
df.loc[is_region, 'Country'] = 'unknown'

# Drop the temporary 'Country_clean' column
df.drop(columns=['Country_clean'], inplace=True)

In [16]:
# Regroup regions and correct typos and more general regions
region_grouping = {
    # Ambiguous regions
    'british isles': 'northern europe',
    'ocean': 'unknown',
    'coast of africa': 'africa',
    'mid-pacifc ocean': 'pacific ocean',  # Corrected typo
    'between portugal & india': 'unknown',
    'the balkans': 'southeast europe',
    'mid atlantic ocean': 'atlantic ocean',
    'central pacific': 'pacific ocean',
    
    # Optional: Group smaller regions under continents
    'caribbean sea': 'caribbean',
    'bay of bengal': 'indian ocean',
    'gulf of aden': 'indian ocean',
    'persian gulf': 'middle east',
    'tasman sea': 'pacific ocean',
    'south china sea': 'pacific ocean',
    'mediterranean sea': 'mediterranean',
}
# Step 1: Correct typos (e.g., "mid-pacifc ocean" → "mid-pacific ocean")
df['Region'] = df['Region'].str.replace('pacifc', 'pacific')

# Step 2: Apply grouping
df['Region'] = df['Region'].replace(region_grouping)

# Step 3: Handle remaining ambiguous entries (e.g., "africa", "asia")
# Optional: If you want continents as regions, keep them. Otherwise:
continent_to_region = {
    'africa': 'africa',  # Keep if continents are valid regions for your analysis
    'asia': 'asia',
    'europe': 'europe',
    'north america': 'north america',
    'south america': 'south america',
}
df['Region'] = df['Region'].replace(continent_to_region)

In [17]:
# Get current column list
cols = df.columns.tolist()

# Remove 'Region' from its current position
cols.remove('Region')

# Find the position of 'Country' and insert 'Region' before it
country_position = cols.index('Country')
cols.insert(country_position, 'Region')

# Reorder the DataFrame
df = df[cols]

-------
Cleaning more the Country column

In [18]:
# 2nd version of country mapping (after taking out the regions)
country_mapping = {
    # Fix typos and variations
    'st kitts / nevis': 'saint kitts and nevis',
    'st. martin': 'saint martin',
    'st. maartin': 'saint martin',
    'st helena, british overseas territory': 'saint helena',
    'turks & caicos': 'turks and caicos',
    'ceylon (sri lanka)': 'sri lanka',
    'ceylon': 'sri lanka',
    'british isles': 'united kingdom',  # Preserve as a distinct entity
    'burma': 'myanmar',
    'western samoa': 'samoa',
    'british new guinea': 'papua new guinea',
    'andaman / nicobar islandas': 'andaman and nicobar islands',
    'andaman islands': 'andaman and nicobar islands',
    'johnston island': 'johnston atoll',  # Preserve as a distinct entity
    'british virgin islands': 'virgin islands',  # Preserve as a distinct entity
    'netherlands antilles': 'netherlands antilles',  # Preserve as a distinct entity
    'san domingo': 'dominican republic',
    'grand cayman': 'cayman islands',  # Preserve as a distinct entity
    'coast of africa': 'africa',  # Preserve as a distinct entity
    'red sea / indian ocean': 'indian ocean',  # Preserve as a distinct entity
    # 'between portugal & india': 'between portugal and india',  # Preserve as a distinct entity
    # 'ocean': 'ocean',  # Preserve as a distinct entity
    # 'unknown': 'unknown',

    # Fix misspellings without grouping
    'columbia': 'colombia',
    
    'united arab emirates (uae)': 'united arab emirates',
    'solomon islands / vanuatu': 'solomon islands and vanuatu',
    'trinidad & tobago': 'trinidad and tobago',
                                                # Countries with double country 
    'egypt / israel': 'egypt',           # Egypt is often in total
    'equatorial guinea / cameroon': 'equatorial guinea',
    'iran / iraq': 'iran',
    'italy / croatia': 'italy',
    # 'red sea / indian ocean': 'red sea and indian ocean',
    # 'mid-pacifc ocean': 'mid-pacific ocean',
    # 'the balkans': 'the balkans',  # Preserve as a distinct entity
}

# Apply the mapping to the 'Country' column
df['Country'] = df['Country'].replace(country_mapping)

print(df['Country'].unique())

['australia' 'bahamas' 'turks and caicos' 'mozambique' 'usa'
 'new caledonia' 'egypt' 'thailand' 'new zealand' 'maldives' 'hawaii'
 'honduras' 'indonesia' 'morocco' 'jamaica' 'belize' 'maldive islands'
 'french polynesia' 'tobago' 'india' 'trinidad' 'south africa' 'mexico'
 'philippines' 'unknown' 'spain' 'portugal' 'samoa' 'colombia' 'ecuador'
 'cuba' 'brazil' 'seychelles' 'argentina' 'fiji' 'england' 'japan'
 'costa rica' 'british overseas territory' 'canada' 'jordan'
 'saint kitts and nevis' 'st martin' 'papua new guinea' 'reunion island'
 'israel' 'china' 'ireland' 'italy' 'malaysia' 'libya' 'mauritius'
 'solomon islands' 'saint helena' 'comoros' 'reunion' 'united kingdom'
 'united arab emirates' 'cape verde' 'dominican republic' 'cayman islands'
 'aruba' 'puerto rico' 'greece' 'saint martin' 'france'
 'trinidad and tobago' 'kiribati' 'diego garcia' 'taiwan'
 'palestinian territories' 'guam' 'nigeria' 'tonga' 'scotland' 'croatia'
 'saudi arabia' 'chile' 'antigua' 'kenya' 'russia' '

In [20]:
region_mapping = {
    # Australia and Oceania
    'australia': 'australia and oceania',
    'new caledonia': 'australia and oceania',
    'new zealand': 'australia and oceania',
    'fiji': 'australia and oceania',
    'samoa': 'australia and oceania',
    'papua new guinea': 'australia and oceania',
    'solomon islands': 'australia and oceania',
    'vanuatu': 'australia and oceania',
    'kiribati': 'australia and oceania',
    'tonga': 'australia and oceania',
    'micronesia': 'australia and oceania',
    'marshall islands': 'australia and oceania',
    'palau': 'australia and oceania',
    'cook islands': 'australia and oceania',
    'french polynesia': 'australia and oceania',
    'american samoa': 'australia and oceania',
    'guam': 'australia and oceania',
    'northern mariana islands': 'australia and oceania',
    'new guinea': 'australia and oceania',
    'admiralty islands': 'australia and oceania',
    'johnston atoll': 'australia and oceania',

    # Caribbean
    'bahamas': 'caribbean',
    'turks and caicos': 'caribbean',
    'jamaica': 'caribbean',
    'cuba': 'caribbean',
    'dominican republic': 'caribbean',
    'cayman islands': 'caribbean',
    'aruba': 'caribbean',
    'puerto rico': 'caribbean',
    'saint kitts and nevis': 'caribbean',
    'saint martin': 'caribbean',
    'trinidad and tobago': 'caribbean',
    'barbados': 'caribbean',
    'grenada': 'caribbean',
    'antigua': 'caribbean',
    'virgin islands': 'caribbean',
    'haiti': 'caribbean',
    'belize': 'caribbean',
    'netherlands antilles': 'caribbean',
    'curacao': 'caribbean',

    # North America
    'usa': 'north america',
    'canada': 'north america',
    'mexico': 'north america',
    'greenland': 'north america',
    'bermuda': 'north america',

    # South America
    'brazil': 'south america',
    'argentina': 'south america',
    'colombia': 'south america',
    'ecuador': 'south america',
    'chile': 'south america',
    'peru': 'south america',
    'venezuela': 'south america',
    'uruguay': 'south america',
    'paraguay': 'south america',
    'guyana': 'south america',
    'falkland islands': 'south america',

    # Europe
    'spain': 'southern europe',
    'portugal': 'southern europe',
    'greece': 'southern europe',
    'italy': 'southern europe',
    'malta': 'southern europe',
    'croatia': 'southern europe',
    'montenegro': 'southern europe',
    'slovenia': 'southern europe',
    'france': 'western europe',
    'united kingdom': 'western europe',
    'ireland': 'western europe',
    'iceland': 'northern europe',
    'norway': 'northern europe',
    'sweden': 'northern europe',
    'azores': 'western europe',
    'cyprus': 'southern europe',
    'crete': 'southern europe',

    # Africa
    'egypt': 'northern africa',
    'morocco': 'northern africa',
    'libya': 'northern africa',
    'tunisia': 'northern africa',
    'algeria': 'northern africa',
    'south africa': 'southern africa',
    'mozambique': 'southern africa',
    'madagascar': 'southern africa',
    'mauritius': 'southern africa',
    'seychelles': 'southern africa',
    'comoros': 'southern africa',
    'reunion': 'southern africa',
    'reunion island': 'southern africa',
    'saint helena': 'southern africa',
    'angola': 'central africa',
    'cameroon': 'central africa',
    'gabon': 'central africa',
    'equatorial guinea': 'central africa',
    'kenya': 'eastern africa',
    'tanzania': 'eastern africa',
    'somalia': 'eastern africa',
    'ethiopia': 'eastern africa',
    'djibouti': 'eastern africa',
    'senegal': 'western africa',
    'nigeria': 'western africa',
    'ghana': 'western africa',
    'liberia': 'western africa',
    'sierra leone': 'western africa',
    'cape verde': 'western africa',
    'namibia': 'southern africa',
    'sudan': 'northern africa',
    'sudan?': 'northern africa',

    # Asia
    'india': 'south asia',
    'sri lanka': 'south asia',
    'maldives': 'south asia',
    'maldive islands': 'south asia',
    'bangladesh': 'south asia',
    'pakistan': 'south asia',
    'iran': 'middle east',
    'iraq': 'middle east',
    'saudi arabia': 'middle east',
    'yemen': 'middle east',
    'jordan': 'middle east',
    'lebanon': 'middle east',
    'syria': 'middle east',
    'kuwait': 'middle east',
    'united arab emirates': 'middle east',
    'qatar': 'middle east',
    'oman': 'middle east',
    'israel': 'middle east',
    'palestinian territories': 'middle east',
    'turkey': 'middle east',
    'china': 'east asia',
    'japan': 'east asia',
    'south korea': 'east asia',
    'north korea': 'east asia',
    'taiwan': 'east asia',
    'hong kong': 'east asia',
    'thailand': 'southeast asia',
    'malaysia': 'southeast asia',
    'singapore': 'southeast asia',
    'indonesia': 'southeast asia',
    'philippines': 'southeast asia',
    'vietnam': 'southeast asia',
    'myanmar': 'southeast asia',
    'cambodia': 'southeast asia',
    'laos': 'southeast asia',
    'brunei': 'southeast asia',

    # Pacific
    'hawaii': 'pacific',
    'okinawa': 'pacific',
    'new britain': 'pacific',
    'java': 'pacific',
    'solomon islands and vanuatu': 'pacific',
    'federated states of micronesia': 'pacific',

    # Other
    'unknown': 'unknown',
    'indian ocean': 'indian ocean',
    'northern arabian sea': 'indian ocean',
    'north sea': 'north sea',
    'atlantic ocean': 'atlantic ocean',
    'pacific ocean': 'pacific ocean',
    'mediterranean sea': 'mediterranean',
}

# Apply the mapping to create/update the 'Region' column
df['Region'] = df['Country'].map(region_mapping)

# Fill any missing values with 'unknown'
df['Region'] = df['Region'].fillna('unknown')

----
# Guillermo Working code:

In [6]:
print('different values in column fatal y/n:\n', df['Fatal Y/N'].value_counts())

unique_values = df['Fatal Y/N'].unique()
print(unique_values)

num_nan_in_column = df['Fatal Y/N'].isna().sum()
print(f"Number of NaN values in 'Fatal Y/N' column: {num_nan_in_column}")

different values in column fatal y/n:
 Fatal Y/N
N    4887
Y    1476
U     641
Name: count, dtype: int64
['N' 'Y' 'U']
Number of NaN values in 'Fatal Y/N' column: 0


In [ ]:
'''# Create a list of regions (oceans, seas, and other non-country entries)
regions = [
    'atlantic ocean', 'pacific ocean', 'indian ocean', 'red sea', 'mediterranean sea',
    'caribbean sea', 'bay of bengal', 'gulf of aden', 'persian gulf', 'tasman sea',
    'south china sea', 'north atlantic ocean', 'south atlantic ocean', 'north pacific ocean',
    'south pacific ocean', 'southwest pacific ocean', 'central pacific', 'coral sea',
    'mid atlantic ocean', 'mid-pacifc ocean', 'between portugal & india', 'coast of africa',
    'the balkans', 'british isles', 'west indies', 'antarctica', 'ocean', 'asia', 'africa',
    'europe', 'north america', 'south america', 'oceania', 'middle east'
]

# Create the 'Region' column
df['Region'] = df['Country'].apply(lambda x: x if x in regions else 'unknown')'''

In [ ]:
country_mapping = {
    # Fix typos and variations
    'st kitts / nevis': 'saint kitts and nevis',
    'st. martin': 'saint martin',
    'st. maartin': 'saint martin',
    'st helena, british overseas territory': 'saint helena',
    'turks & caicos': 'turks and caicos',
    'ceylon (sri lanka)': 'sri lanka',
    'ceylon': 'sri lanka',
    'burma': 'myanmar',
    'western samoa': 'samoa',
    'british new guinea': 'papua new guinea',
    'andaman / nicobar islandas': 'andaman and nicobar islands',
    'andaman islands': 'andaman and nicobar islands',
    'johnston island': 'johnston atoll',  # Preserve as a distinct entity
    'british virgin islands': 'virgin islands',  # Preserve as a distinct entity
    'netherlands antilles': 'netherlands antilles',  # Preserve as a distinct entity
    'san domingo': 'dominican republic',
    'grand cayman': 'cayman islands',  # Preserve as a distinct entity
    'british isles': 'british isles',  # Preserve as a distinct entity
    'the balkans': 'the balkans',  # Preserve as a distinct entity
    'coast of africa': 'coast of africa',  # Preserve as a distinct entity
    'between portugal & india': 'between portugal and india',  # Preserve as a distinct entity
    'ocean': 'ocean',  # Preserve as a distinct entity
    'unknown': 'unknown',

    # Fix misspellings without grouping
    'columbia': 'colombia',
    'equatorial guinea / cameroon': 'equatorial guinea and cameroon',
    'mid-pacifc ocean': 'mid-pacific ocean',
    'red sea / indian ocean': 'red sea and indian ocean',
    'egypt / israel': 'egypt and israel',
    'solomon islands / vanuatu': 'solomon islands and vanuatu',
    'iran / iraq': 'iran and iraq',
    'italy / croatia': 'italy and croatia',
    'trinidad & tobago': 'trinidad and tobago',
    'united arab emirates (uae)': 'united arab emirates',
}

In [19]:
'''# Create a mapping dictionary for standardization
country_mapping = {
    'ceylon (sri lanka)': 'sri lanka',
    'st. martin': 'saint martin',
    'st helena, british overseas territory': 'saint helena',
    'st kitts / nevis': 'saint kitts and nevis',
    'st. maartin': 'saint martin',
    'usa': 'united states',
    'british overseas territory': 'united kingdom',  # Adjust based on context
    'british virgin islands': 'virgin islands',  # Adjust based on context
    'british west indies': 'west indies',
    'reunion island': 'reunion',
    'united arab emirates (uae)': 'united arab emirates',
    'andaman / nicobar islandas': 'andaman and nicobar islands',
    'mid atlantic ocean': 'atlantic ocean',
    'mid-pacifc ocean': 'pacific ocean',
    'red sea / indian ocean': 'indian ocean',
    'egypt / israel': 'egypt',  # Choose one or keep as is
    'equatorial guinea / cameroon': 'equatorial guinea',  # Choose one or keep as is
    'solomon islands / vanuatu': 'solomon islands',  # Choose one or keep as is
    'iran / iraq': 'iran',  # Choose one or keep as is
    'italy / croatia': 'italy',  # Choose one or keep as is
    'trinidad & tobago': 'trinidad and tobago',
    'turks & caicos': 'turks and caicos',
    'western samoa': 'samoa',
    'bay of bengal': 'indian ocean',  # Region, not a country
    'caribbean sea': 'caribbean',  # Region, not a country
    'mediterranean sea': 'mediterranean',  # Region, not a country
    'gulf of aden': 'indian ocean',  # Region, not a country
    'persian gulf': 'indian ocean',  # Region, not a country
    'tasman sea': 'pacific ocean',  # Region, not a country
    'south china sea': 'pacific ocean',  # Region, not a country
    'north atlantic ocean': 'atlantic ocean',  # Region, not a country
    'south atlantic ocean': 'atlantic ocean',  # Region, not a country
    'north pacific ocean': 'pacific ocean',  # Region, not a country
    'south pacific ocean': 'pacific ocean',  # Region, not a country
    'southwest pacific ocean': 'pacific ocean',  # Region, not a country
    'central pacific': 'pacific ocean',  # Region, not a country
    'coral sea': 'pacific ocean',  # Region, not a country
    'indian ocean?': 'indian ocean',  # Region, not a country
    'red sea?': 'red sea',  # Region, not a country
    'between portugal & india': 'indian ocean',  # Region, not a country
    'coast of africa': 'africa',  # Region, not a country
    'the balkans': 'europe',  # Region, not a country
    'british isles': 'united kingdom',  # Region, not a country
    'west indies': 'caribbean',  # Region, not a country
    'admiralty islands': 'papua new guinea',  # Part of Papua New Guinea
    'andaman islands': 'andaman and nicobar islands',  # Part of India
    'johnston island': 'united states',  # US territory
    'guam': 'united states',  # US territory
    'puerto rico': 'united states',  # US territory
    'american samoa': 'united states',  # US territory
    'northern mariana islands': 'united states',  # US territory
    'french polynesia': 'france',  # French territory
    'new caledonia': 'france',  # French territory
    'mayotte': 'france',  # French territory
    'reunion': 'france',  # French territory
    'martinique': 'france',  # French territory
    'netherlands antilles': 'netherlands',  # Dutch territory
    'curacao': 'netherlands',  # Dutch territory
    'aruba': 'netherlands',  # Dutch territory
    'bermuda': 'united kingdom',  # British territory
    'cayman islands': 'united kingdom',  # British territory
    'falkland islands': 'united kingdom',  # British territory
    'gibraltar': 'united kingdom',  # British territory
    'montserrat': 'united kingdom',  # British territory
    'pitcairn islands': 'united kingdom',  # British territory
    'saint helena': 'united kingdom',  # British territory
    'south georgia and the south sandwich islands': 'united kingdom',  # British territory
    'turks and caicos islands': 'united kingdom',  # British territory
    'virgin islands': 'united kingdom',  # British territory
    'greenland': 'denmark',  # Danish territory
    'faroe islands': 'denmark',  # Danish territory
    'aland islands': 'finland',  # Finnish territory
    'svalbard and jan mayen': 'norway',  # Norwegian territory
    'bouvet island': 'norway',  # Norwegian territory
    'heard island and mcdonald islands': 'australia',  # Australian territory
    'cocos (keeling) islands': 'australia',  # Australian territory
    'christmas island': 'australia',  # Australian territory
    'norfolk island': 'australia',  # Australian territory
    'ashmore and cartier islands': 'australia',  # Australian territory
    'coral sea islands': 'australia',  # Australian territory
    'antarctica': 'antarctica',  # Not a country
    'ocean': 'unknown',  # Unknown
    'asia?': 'unknown',  # Unknown
    'africa': 'unknown',  # Unknown
    'atlantic ocean': 'unknown',  # Unknown
    'pacific ocean': 'unknown',  # Unknown
    'indian ocean': 'unknown',  # Unknown
    'red sea': 'unknown',  # Unknown
    'mediterranean': 'unknown',  # Unknown
    'caribbean': 'unknown',  # Unknown
    'europe': 'unknown',  # Unknown
}

# Apply the mapping to the 'Country' column
df['Country'] = df['Country'].replace(country_mapping)'''

"# Create a mapping dictionary for standardization\ncountry_mapping = {\n    'ceylon (sri lanka)': 'sri lanka',\n    'st. martin': 'saint martin',\n    'st helena, british overseas territory': 'saint helena',\n    'st kitts / nevis': 'saint kitts and nevis',\n    'st. maartin': 'saint martin',\n    'usa': 'united states',\n    'british overseas territory': 'united kingdom',  # Adjust based on context\n    'british virgin islands': 'virgin islands',  # Adjust based on context\n    'british west indies': 'west indies',\n    'reunion island': 'reunion',\n    'united arab emirates (uae)': 'united arab emirates',\n    'andaman / nicobar islandas': 'andaman and nicobar islands',\n    'mid atlantic ocean': 'atlantic ocean',\n    'mid-pacifc ocean': 'pacific ocean',\n    'red sea / indian ocean': 'indian ocean',\n    'egypt / israel': 'egypt',  # Choose one or keep as is\n    'equatorial guinea / cameroon': 'equatorial guinea',  # Choose one or keep as is\n    'solomon islands / vanuatu': 'sol

In [ ]:
# 1st variation of country mapping  (too many and lots of errors)
# Create a mapping dictionary for standardization
country_mapping = {
    'ceylon (sri lanka)': 'sri lanka',
    'st. martin': 'saint martin',
    'st helena, british overseas territory': 'saint helena',
    'st kitts / nevis': 'saint kitts and nevis',
    'st. maartin': 'saint martin',
    'usa': 'united states',
    'british overseas territory': 'united kingdom',  # Adjust based on context
    'british virgin islands': 'virgin islands',  # Adjust based on context
    'british west indies': 'west indies',
    'reunion island': 'reunion',
    'united arab emirates (uae)': 'united arab emirates',
    'andaman / nicobar islandas': 'andaman and nicobar islands',
    'mid atlantic ocean': 'atlantic ocean',
    'mid-pacifc ocean': 'pacific ocean',
    'red sea / indian ocean': 'indian ocean',
    'egypt / israel': 'egypt',  # Choose one or keep as is
    'equatorial guinea / cameroon': 'equatorial guinea',  # Choose one or keep as is
    'solomon islands / vanuatu': 'solomon islands',  # Choose one or keep as is
    'iran / iraq': 'iran',  # Choose one or keep as is
    'italy / croatia': 'italy',  # Choose one or keep as is
    'trinidad & tobago': 'trinidad and tobago',
    'turks & caicos': 'turks and caicos',
    'western samoa': 'samoa',
    'bay of bengal': 'indian ocean',  # Region, not a country
    'caribbean sea': 'caribbean',  # Region, not a country
    'mediterranean sea': 'mediterranean',  # Region, not a country
    'gulf of aden': 'indian ocean',  # Region, not a country
    'persian gulf': 'indian ocean',  # Region, not a country
    'tasman sea': 'pacific ocean',  # Region, not a country
    'south china sea': 'pacific ocean',  # Region, not a country
    'north atlantic ocean': 'atlantic ocean',  # Region, not a country
    'south atlantic ocean': 'atlantic ocean',  # Region, not a country
    'north pacific ocean': 'pacific ocean',  # Region, not a country
    'south pacific ocean': 'pacific ocean',  # Region, not a country
    'southwest pacific ocean': 'pacific ocean',  # Region, not a country
    'central pacific': 'pacific ocean',  # Region, not a country
    'coral sea': 'pacific ocean',  # Region, not a country
    'indian ocean?': 'indian ocean',  # Region, not a country
    'red sea?': 'red sea',  # Region, not a country
    'between portugal & india': 'indian ocean',  # Region, not a country
    'coast of africa': 'africa',  # Region, not a country
    'the balkans': 'europe',  # Region, not a country
    'british isles': 'united kingdom',  # Region, not a country
    'west indies': 'caribbean',  # Region, not a country
    'admiralty islands': 'papua new guinea',  # Part of Papua New Guinea
    'andaman islands': 'andaman and nicobar islands',  # Part of India
    'johnston island': 'united states',  # US territory
    'guam': 'united states',  # US territory
    'puerto rico': 'united states',  # US territory
    'american samoa': 'united states',  # US territory
    'northern mariana islands': 'united states',  # US territory
    'french polynesia': 'france',  # French territory
    'new caledonia': 'france',  # French territory
    'mayotte': 'france',  # French territory
    'reunion': 'france',  # French territory
    'martinique': 'france',  # French territory
    'netherlands antilles': 'netherlands',  # Dutch territory
    'curacao': 'netherlands',  # Dutch territory
    'aruba': 'netherlands',  # Dutch territory
    'bermuda': 'united kingdom',  # British territory
    'cayman islands': 'united kingdom',  # British territory
    'falkland islands': 'united kingdom',  # British territory
    'gibraltar': 'united kingdom',  # British territory
    'montserrat': 'united kingdom',  # British territory
    'pitcairn islands': 'united kingdom',  # British territory
    'saint helena': 'united kingdom',  # British territory
    'south georgia and the south sandwich islands': 'united kingdom',  # British territory
    'turks and caicos islands': 'united kingdom',  # British territory
    'virgin islands': 'united kingdom',  # British territory
    'greenland': 'denmark',  # Danish territory
    'faroe islands': 'denmark',  # Danish territory
    'aland islands': 'finland',  # Finnish territory
    'svalbard and jan mayen': 'norway',  # Norwegian territory
    'bouvet island': 'norway',  # Norwegian territory
    'heard island and mcdonald islands': 'australia',  # Australian territory
    'cocos (keeling) islands': 'australia',  # Australian territory
    'christmas island': 'australia',  # Australian territory
    'norfolk island': 'australia',  # Australian territory
    'ashmore and cartier islands': 'australia',  # Australian territory
    'coral sea islands': 'australia',  # Australian territory
    'antarctica': 'antarctica',  # Not a country
    
}

# Apply the mapping to the 'Country' column
df['Country'] = df['Country'].replace(country_mapping)

In [24]:
df['Country'].value_counts()


Country
usa                            2556
australia                      1497
south africa                    598
unknown                         160
new zealand                     146
                               ... 
netherlands antilles              1
northern mariana islands          1
java                              1
solomon islands and vanuatu       1
korea                             1
Name: count, Length: 168, dtype: int64

In [72]:
# Country cleaning summary

df['Country'].value_counts()
unique_countries = df['Country'].unique()
print("Unique country values:", unique_countries)

number_of_unique_countries = df['Country'].nunique()
print("Number of unique countries:", number_of_unique_countries)

'''
Countries before cleaning:
Number of unique countries: 241

Countries after first cleaning step:

Name: count, Length: 211, dtype: int64
array(['australia', 'bahamas', 'turks and caicos', 'mozambique', 'usa',
       'new caledonia', 'egypt', 'thailand', 'new zealand', 'maldives',
       'hawaii', 'honduras', 'indonesia', 'morocco', 'jamaica', 'belize',
       'maldive islands', 'french polynesia', 'tobago', 'india',
       'trinidad', 'south africa', 'mexico', 'philippines', 'coral sea',
       'spain', 'portugal', 'samoa', 'colombia', 'ecuador', 'cuba',
       'brazil', 'seychelles', 'argentina', 'fiji', 'england', 'japan',
       'columbia', 'costa rica', 'british overseas territory', 'canada',
       'jordan', 'st kitts / nevis', 'st martin', 'papua new guinea',
       'reunion island', 'israel', 'china', 'ireland', 'italy',
       'malaysia', 'libya', nan, 'mauritius', 'solomon islands',
       'st helena, british overseas territory', 'comoros', 'reunion',
       'united kingdom', 'united arab emirates', 'cape verde',
       'dominican republic', 'cayman islands', 'aruba', 'puerto rico',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'trinidad & tobago', 'kiribati', 'diego garcia', 'taiwan',
       'palestinian territories', 'guam', 'nigeria', 'tonga', 'scotland',
       'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya', 'russia',
       'turks & caicos', 'united arab emirates (uae)', 'azores',
       'south korea', 'malta', 'vietnam', 'madagascar', 'panama',
       'somalia', 'nevis', 'british virgin islands', 'norway', 'senegal',
       'yemen', 'gulf of aden', 'sierra leone', 'st. maartin',
       'grand cayman', 'liberia', 'vanuatu', 'venezuela', 'sri lanka',
       'uruguay', 'micronesia', 'caribbean sea', 'okinawa', 'tanzania',
       'marshall islands', 'egypt / israel', 'northern arabian sea',
       'hong kong', 'el salvador', 'angola', 'bermuda', 'montenegro',
       'iran', 'tunisia', 'namibia', 'north atlantic ocean',
       'south china sea', 'bangladesh', 'palau', 'western samoa',
       'pacific ocean', 'british isles', 'grenada', 'iraq', 'turkey',
       'singapore', 'new britain', 'sudan', 'johnston island',
       'south pacific ocean', 'new guinea', 'red sea',
       'north pacific ocean', 'federated states of micronesia',
       'mid atlantic ocean', 'admiralty islands', 'british west indies',
       'south atlantic ocean', 'persian gulf', 'red sea / indian ocean',
       'north sea', 'nicaragua', 'american samoa',
       'andaman / nicobar islandas', 'gabon', 'mayotte', 'the balkans',
       'sudan?', 'martinique', 'indian ocean', 'guatemala',
       'netherlands antilles', 'northern mariana islands', 'iran / iraq',
       'java', 'central pacific', 'solomon islands / vanuatu',
       'southwest pacific ocean', 'bay of bengal', 'mid-pacifc ocean',
       'slovenia', 'curacao', 'iceland', 'italy / croatia', 'barbados',
       'monaco', 'guyana', 'haiti', 'san domingo', 'kuwait',
       'falkland islands', 'crete', 'cyprus', 'west indies', 'burma',
       'lebanon', 'paraguay', 'british new guinea', 'ceylon', 'ocean',
       'georgia', 'syria', 'tuvalu', 'indian ocean?', 'guinea',
       'andaman islands', 'equatorial guinea / cameroon', 'cook islands',
       'peru', 'africa', 'algeria', 'coast of africa', 'tasman sea',
       'ghana', 'greenland', 'mediterranean sea', 'sweden', 'roatan',
       'between portugal & india', 'djibouti', 'bahrein', 'korea',
       'red sea?', 'asia?', 'ceylon (sri lanka)'], dtype=object)

'''

Unique country values: ['australia' 'bahamas' 'turks and caicos' 'mozambique' 'usa'
 'new caledonia' 'egypt' 'thailand' 'new zealand' 'maldives' 'hawaii'
 'honduras' 'indonesia' 'morocco' 'jamaica' 'belize' 'maldive islands'
 'french polynesia' 'tobago' 'india' 'trinidad' 'south africa' 'mexico'
 'philippines' 'unknown' 'spain' 'portugal' 'samoa' 'colombia' 'ecuador'
 'cuba' 'brazil' 'seychelles' 'argentina' 'fiji' 'england' 'japan'
 'costa rica' 'british overseas territory' 'canada' 'jordan'
 'saint kitts and nevis' 'st martin' 'papua new guinea' 'reunion island'
 'israel' 'china' 'ireland' 'italy' 'malaysia' 'libya' 'mauritius'
 'solomon islands' 'saint helena' 'comoros' 'reunion' 'united kingdom'
 'united arab emirates' 'cape verde' 'dominican republic' 'cayman islands'
 'aruba' 'puerto rico' 'greece' 'saint martin' 'france'
 'trinidad and tobago' 'kiribati' 'diego garcia' 'taiwan'
 'palestinian territories' 'guam' 'nigeria' 'tonga' 'scotland' 'croatia'
 'saudi arabia' 'chile' 'anti

"\nCountries before cleaning:\nNumber of unique countries: 241\n\nCountries after first cleaning step:\n\nName: count, Length: 211, dtype: int64\narray(['australia', 'bahamas', 'turks and caicos', 'mozambique', 'usa',\n       'new caledonia', 'egypt', 'thailand', 'new zealand', 'maldives',\n       'hawaii', 'honduras', 'indonesia', 'morocco', 'jamaica', 'belize',\n       'maldive islands', 'french polynesia', 'tobago', 'india',\n       'trinidad', 'south africa', 'mexico', 'philippines', 'coral sea',\n       'spain', 'portugal', 'samoa', 'colombia', 'ecuador', 'cuba',\n       'brazil', 'seychelles', 'argentina', 'fiji', 'england', 'japan',\n       'columbia', 'costa rica', 'british overseas territory', 'canada',\n       'jordan', 'st kitts / nevis', 'st martin', 'papua new guinea',\n       'reunion island', 'israel', 'china', 'ireland', 'italy',\n       'malaysia', 'libya', nan, 'mauritius', 'solomon islands',\n       'st helena, british overseas territory', 'comoros', 'reunion',\n    

In [30]:
australia_rows = df.loc[df['Region'] == 'ocean']
display(australia_rows)


,Date,Year,Type,Region,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
6441,1887,1887.0,Sea Disaster,ocean,unknown,"Somewhere between Philadelphia and Hiogo, Japan",NaN,British ship Macedon was thrown on her beam en...,a ship's steward,M,NaN,Foot severed,N,NaN,NaN,"Galveston Daily News, 2/11/1888",1887.00.00-Macedom.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1887.00.00,1887.00.00,562.0,NaN,NaN


In [73]:
# Extract unique country values
unique_countries = df['Country'].unique()

unique_countries_without_nan = [country for country in unique_countries if isinstance(country, str)]

# Sort the list
sorted_unique_countries = sorted(unique_countries_without_nan)

# Print the sorted list of unique countries
print("Sorted unique country values:", sorted_unique_countries)


Sorted unique country values: ['admiralty islands', 'algeria', 'american samoa', 'andaman and nicobar islands', 'angola', 'antigua', 'argentina', 'aruba', 'australia', 'azores', 'bahamas', 'bahrein', 'bangladesh', 'barbados', 'belize', 'bermuda', 'brazil', 'british overseas territory', 'british west indies', 'canada', 'cape verde', 'cayman islands', 'chile', 'china', 'colombia', 'comoros', 'cook islands', 'costa rica', 'crete', 'croatia', 'cuba', 'curacao', 'cyprus', 'diego garcia', 'djibouti', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'england', 'equatorial guinea', 'falkland islands', 'federated states of micronesia', 'fiji', 'france', 'french polynesia', 'gabon', 'georgia', 'ghana', 'greece', 'greenland', 'grenada', 'guam', 'guatemala', 'guinea', 'guyana', 'haiti', 'hawaii', 'honduras', 'hong kong', 'iceland', 'india', 'indian ocean', 'indonesia', 'iran', 'iraq', 'ireland', 'israel', 'italy', 'jamaica', 'japan', 'java', 'johnston atoll', 'jordan', 'kenya', 'kiribati',

In [67]:
# Count the frequency of each value in the 'Country' column
country_counts = df['Country'].value_counts()

# Display the result
print(country_counts)

# Check how many times 'israel' appears
israel_count = country_counts.loc['italy']
print(f"appears {israel_count} times.")

Country
usa                               2556
australia                         1497
south africa                       598
unknown                            160
new zealand                        146
                                  ... 
admiralty islands                    1
federated states of micronesia       1
western samoa                        1
bangladesh                           1
ceylon (sri lanka)                   1
Name: count, Length: 182, dtype: int64
appears 72 times.
